# PoliGPT - Conversação RAG

In [ ]:
import os
import json
from typing import Any, Dict, List, Optional, Tuple

os.environ["AZURESEARCH_FIELDS_ID"] = "chunk_id"
os.environ["AZURESEARCH_FIELDS_CONTENT"] = "chunk"
os.environ["AZURESEARCH_FIELDS_CONTENT_VECTOR"] = "vector"
os.environ["AZURESEARCH_FIELDS_TAG"] = None

In [ ]:
from langchain_core.utils import get_from_env
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_community.vectorstores.azuresearch import AzureSearch, AzureSearchVectorStoreRetriever
from langchain_openai import OpenAIEmbeddings

In [3]:
from langchain.tools.retriever import create_retriever_tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [4]:
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential
from azure.keyvault.secrets import SecretClient

In [18]:
AZURESEARCH_FIELDS_CONTENT_VECTOR = os.getenv("AZURESEARCH_FIELDS_CONTENT_VECTOR")
print(AZURESEARCH_FIELDS_CONTENT_VECTOR)

vector


## Key Vault Client

In [5]:
kv_uri = "https://kv-poligpt-dev-eastus2.vault.azure.net"
credential = DefaultAzureCredential()

client = SecretClient(vault_url=kv_uri, credential=credential)

## Parameters

In [6]:
COMPLETION_MODEL = "gpt-4o-2024-08-06"
EMBEDDING_MODEL = "text-embedding-3-large"
EMBEDDING_DIMENSIONS = 3072

OPENAI_API_KEY = client.get_secret("openai-api-key").value

In [7]:
AZURE_SEARCH_ENDPOINT = client.get_secret("azure-search-endpoint").value
AZURE_SEARCH_ADMIN_KEY = client.get_secret("azure-search-admin-key").value
SEARCH_INDEX_NAME = "poligpt-index"

## Setup

### LLM

### Embeddings

In [8]:
embeddings = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY, 
    model=EMBEDDING_MODEL
)

### Vector Store - Azure Search AI

In [ ]:
class CustomAzureSearchRetriever(BaseRetriever):
    """
    Essa classe implementa um Retriever para Azure Search AI,
    sobrescrevendo o método _get_relevant_documents, para tornar o output dos resultados das 
    buscas mais compacto sem necessidade de alterar a estrutura de campos no Index Search AI. 
    Isso irá favorecer o uso desses resultados dentro do contexto de Retrieval Augmented
    Generation, pois irá evitar uma quantidade desnecessária de tokens extras.

    """
    def __init__(
        self, k: int = 5,
        search_type: str = "hybrid",
        id_field: str,
        content_field: str,
        vector_field: str,
        extra_fields: List = [],
        **kwargs: Any
    ):     
        super().__init__(**kwargs)

        self.k = k
        self.search_type = search_type
        self.id_field = id_field
        self.content_field = content_field
        self.vector_field = vector_field
        self.extra_fields = extra_fields

    def hybrid_search_with_score(
        self, query: str, k: int = 4, filters: Optional[str] = None, **kwargs: Any,
    ) -> List[Tuple[Document, float]]:
        
        embedding = self.embed_query(query)
        results = self._simple_search(embedding, query, k, filters=filters, **kwargs)

        return _results_to_documents(results)
    
    def _get_relevant_documents(
        self, query: str, run_manager: CallbackManagerForRetrieverRun, **kwargs: Any,
    ) -> List[Document]:
        params = {**self.search_kwargs, **kwargs}

        if self.search_type == "similarity":
            docs = self.vectorstore.vector_search(query, k=self.k, **params)
        elif self.search_type == "hybrid":
            docs = self.vectorstore.hybrid_search(query, k=self.k, **params)
        elif self.search_type == "semantic_hybrid":
            docs = self.vectorstore.semantic_hybrid_search(query, k=self.k, **params)
        else:
            raise ValueError(f"search_type of {self.search_type} not allowed.")
        return docs

In [ ]:
class CustomAzureSearchRetriever(AzureSearchVectorStoreRetriever):
    """
    Essa classe implementa um customização do Retriever LangChain para Azure Search AI,
    sobrescrevendo o método _get_relevant_documents, para tornar o output dos resultados das 
    buscas mais compacto sem necessidade de alterar a estrutura de campos no Index Search AI. 
    Isso irá favorecer o uso desses resultados dentro do contexto de Retrieval Augmented
    Generation, pois irá evitar uma quantidade desnecessária de tokens extras.
    
    Setup:
        As seguintes variáveis de ambiente devem estar definidas:

        AZURESEARCH_FIELDS_ID = "<CHAVE_PRIMARIA_INDEX_AZURE_SEARCH>"
        AZURESEARCH_FIELDS_CONTENT_VECTOR = "<CAMPO_VETOR_AZURE_SEARCH>"
        AZURESEARCH_FIELDS_CONTENT = "<CAMPO_CONTEUDO_TEXTO_AZURE_SEARCH>"
        AZURESEARCH_FIELDS_TAG = "<CAMPO_METADADOS_AZURE_SEARCH>"

        O campo de metadados pode opcionalmente ser deixado vazio, pois não é necessário na
        maioria dos casos de uso.

    """
    def __init__(self, extra_fields: List = [], **kwargs: Any):     
        super().__init__(**kwargs)

        self.extra_fields = extra_fields

    def _get_relevant_documents(
        self,
        query: str,
        run_manager: CallbackManagerForRetrieverRun,
        **kwargs: Any,
    ) -> List[Document]:
        # Fields metadata
        if FIELDS_METADATA in result:
            if isinstance(result[FIELDS_METADATA], dict):
                fields_metadata = result[FIELDS_METADATA]
            else:
                fields_metadata = json.loads(result[FIELDS_METADATA])
        else:
            fields_metadata = {
                key: value for key, value in result.items() if key in extra_fields
            }
        # IDs
        if FIELDS_ID in result:
            fields_id = {FIELDS_ID: result.pop(FIELDS_ID)}
        else:
            fields_id = {}
        return Document(
            page_content=result.pop(FIELDS_CONTENT),
            metadata={
                **fields_id,
                **fields_metadata,
            },
        )

In [ ]:
vector_store = AzureSearch(
    azure_search_endpoint=AZURE_SEARCH_ENDPOINT,
    azure_search_key=AZURE_SEARCH_ADMIN_KEY,
    index_name=SEARCH_INDEX_NAME,
    embedding_function=embeddings.embed_query
)

retriever = AzureSearchVectorStoreRetriever()

In [10]:
docs = vector_store.hybrid_search(
    query="Como me inscrevo numa materia?",
    k=3
)

In [ ]:
docs

[Document(metadata={'chunk_id': '90a4571bb413_7e37d055112566ce114ae3d34b4ec12f_pages_0', 'chunk': 'STI Setor de Tecnologia da Informação \nEscola Politécnica \nUFRJ\n\nPoliMoodle – Inscrição em Disciplina\n\nSTI Poli UFRJ – http://www.sti.poli.ufrj.br – sti@poli.uftj.br V2020-08-19\n\nPara se inscrever como aluno em uma disciplina no PoliMoodle, você deve seguir os passos abaixo:\n\n1. Acesse o ambiente PoliMoodle (http://www.moodle.poli.ufrj.br), informando a sua conta e a sua senha.\n\n2. A área “Meus cursos” mostra as disciplinas que você está inscrito. Clique em "Todos os cursos" para se \ninscrever em outras disciplinas.\n\n3. Na nova tela, você poderá navegar pelos Departamentos para ter acesso às disciplinas disponíveis.\n\nSTI Setor de Tecnologia da Informação \nEscola Politécnica \nUFRJ\n\nPoliMoodle – Inscrição em Disciplina\n\nSTI Poli UFRJ – http://www.sti.poli.ufrj.br – sti@poli.uftj.br V2020-08-19\n\nOutra forma para ter acesso às disciplinas disponíveis é utilizando o re

: 

### Chat History 